# Notebook for End to End testing in the MID PSI

- LMC DISH in the loop
- Receives and plots the visibilities to SDP

In [1]:
import os
import json
from time import sleep
import tango
from PyTango import Database, DeviceProxy
print("Setting up parameters\n")

SUT_NAMESPACE = "ci-ska-mid-psi-1339371391-jaredmda"

TANGO_HOST = f"databaseds-tango-base.{SUT_NAMESPACE}.svc.cluster.local:10000"
os.environ["TANGO_HOST"] = TANGO_HOST

RECEPTORS = ["SKA001"]

# Config files set up
DATA_DIR = "../../data"
TMC_CONFIGS = f"{DATA_DIR}/mid_telescope/tmc"
SCAN_FILE = f"{TMC_CONFIGS}/scan.json"
RELEASE_RESOURCES_FILE = f"{DATA_DIR}/release_resources.json"

ASSIGN_RESOURCES_FILE = f"{TMC_CONFIGS}/assign_resources.json"
CONFIGURE_SCAN_FILE = f"{TMC_CONFIGS}/configure_scan.json"

CBF_CONFIGS = f"{DATA_DIR}/mid_telescope/cbf"
DISH_CONFIG_FILE = f"{CBF_CONFIGS}/sys_params/load_dish_config.json"

# MCS/Talon setup
target_board = [1]
COMMON_CONFIG_FOLDER = os.path.join(DATA_DIR, "mid_telescope/cbf")
HW_CONFIG_FOLDER = os.path.join(COMMON_CONFIG_FOLDER, "hw_config")
INIT_SYS_PARAM_FILE = os.path.join(COMMON_CONFIG_FOLDER, "sys_params/initial_system_param.json")

print("Links for Taranta and QA Display\n")
print("SUT Links")
print(f"https://142.73.34.170/{SUT_NAMESPACE}/signal/display/")
print(f"https://142.73.34.170/{SUT_NAMESPACE}/taranta/devices")

Setting up parameters

Links for Taranta and QA Display

SUT Links
https://142.73.34.170/ci-ska-mid-psi-1339371391-jaredmda/signal/display/
https://142.73.34.170/ci-ska-mid-psi-1339371391-jaredmda/taranta/devices


### MCS Deployer Setup

In [2]:
db = Database()

deployer_tango = DeviceProxy("mid_csp_cbf/ec/deployer")
# Check the devices initially deployed to the database
print(*db.get_device_exported("*").value_string, sep="\n")
# Make sure the deployer device is set to ON
deployer_tango.On()
print(deployer_tango.state())

if any(i > 4 for i in target_board):
    print("Using swap for higher number talons")
    config = "hw_config_swap_psi.yaml"
    print("Modifying target to use lower nums to match swap file")
    target_board = list(map(lambda x: x - 4, target_board))
else:
    print("Using standard HW config")
    config = "hw_config_psi.yaml"

HW_CONFIG_FILE = os.path.join(HW_CONFIG_FOLDER, config)
if os.path.isfile(HW_CONFIG_FILE):
    print("HW config: ✔️")
else:
    print("hw config: ❌")

dserver/CbfController/controller
dserver/CbfSubarray/cbfsubarray
dserver/central_node_mid/01
dserver/csp_master_leaf_node/01
dserver/csp_subarray_leaf_node_mid/01
dserver/csp_subarray_leaf_node_mid/02
dserver/DataBaseds/2
dserver/DishLogger/001
dserver/DishManagerDS/001
dserver/dish_leaf_node/001
dserver/DSManager/001
dserver/ECBite/bite
dserver/ECDeployer/deployer
dserver/FspMulti/fsp-01
dserver/FspMulti/fsp-02
dserver/FspMulti/fsp-03
dserver/FspMulti/fsp-04
dserver/MidCspCapabilityFsp/capabilityfsp
dserver/MidCspCapabilityVcc/capabilityvcc
dserver/MidCspController/controller
dserver/MidCspSubarray/subarray1
dserver/MidCspSubarray/subarray2
dserver/MidCspSubarray/subarray3
dserver/PowerSwitch/pdu
dserver/SDPController/0
dserver/SDPQueueConnector/01
dserver/SDPSubarray/01
dserver/sdp_master_leaf_node/01
dserver/sdp_subarray_leaf_node/01
dserver/sdp_subarray_leaf_node/02
dserver/ska-mid-spfrx-controller-ds/spfrx-20
dserver/ska-mid-spfrx-system-ds/spfrx-20
dserver/ska-talondx-bsp-ds/spfr

In [3]:
!kubectl cp $HW_CONFIG_FILE $SUT_NAMESPACE/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml

In [4]:
deployer_tango.targetTalons = target_board
print(deployer_tango.targetTalons)

[1]


In [5]:
deployer_tango.generate_config_jsons()

In [6]:
deployer_tango.set_timeout_millis(200000)
try:
    deployer_tango.download_artifacts()
except tango.DevFailed as e:
    print(e)
    print(
        "Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes."
    )
deployer_tango.set_timeout_millis(4500)

In [7]:
deployer_tango.configure_db()

### Set Up and Configure Device Proxies 

In [9]:
print("Setting up the device proxies")

# TMC proxies
tmc_central_node = DeviceProxy("ska_mid/tm_central/central_node")
tmc_csp_master = DeviceProxy("ska_mid/tm_leaf_node/csp_master")
tmc_csp_subarray = DeviceProxy("ska_mid/tm_leaf_node/csp_subarray01")
tmc_subarray = DeviceProxy("ska_mid/tm_subarray_node/1")

# CSP.LMC proxies
csp_control = DeviceProxy("mid-csp/control/0")
csp_subarray = DeviceProxy("mid-csp/subarray/01")

# CBF proxies
cbf_controller = DeviceProxy("mid_csp_cbf/sub_elt/controller")
cbf_subarray = DeviceProxy("mid_csp_cbf/sub_elt/subarray_01")

# Dish Leaf Proxies
dish_leaf_node_ska001 = DeviceProxy("ska_mid/tm_leaf_node/d0001")

# SDP Proxies
sdp_subarray = DeviceProxy("mid-sdp/subarray/01")

# Set devices to adminMode = ONLINE
csp_control.adminMode = 0
csp_subarray.adminMode = 0

# Leaf Nodes
csp_subarray_leaf_node = "ska_mid/tm_leaf_node/csp_subarray01"
sdp_subarray_leaf_node = "ska_mid/tm_leaf_node/sdp_subarray01"
csp_master_leaf_node = "ska_mid/tm_leaf_node/csp_master"
csp_subarray_leaf_node_dp = DeviceProxy(csp_subarray_leaf_node)
sdp_subarray_leaf_node_dp = DeviceProxy(sdp_subarray_leaf_node)
csp_master_leaf_node_dp = DeviceProxy(csp_master_leaf_node)

sleep(2)
print("\nChecking admin mode after setting to ONLINE (0):")
print(f"  CSP Control: {csp_control.adminMode}")
print(f"  CSP Subarray: {csp_subarray.adminMode}")
print(f"  CBF Controller: {cbf_controller.adminMode}")
print(f"  CBF Subarray: {cbf_subarray.adminMode}")

# Set CBF Simulation mode to false and CBF timeout to 99s
csp_control.cbfSimulationMode = 0
csp_control.commandTimeout = 99

sleep(2)
print("\nChecking CBF Simulation Mode and CBF Timeout:")
print(f"  CBF Simulation Mode: {bool(csp_control.cbfSimulationMode)}")
print(f"  CBF Timeout: {csp_control.commandTimeout} sec")

Setting up the device proxies

Checking admin mode after setting to ONLINE (0):
  CSP Control: 0
  CSP Subarray: 0
  CBF Controller: 0
  CBF Subarray: 0

Checking CBF Simulation Mode and CBF Timeout:
  CBF Simulation Mode: False
  CBF Timeout: 99 sec


### Load the Dish Vcc Config / Init Sys Params

NOTE: RUN TWICE

In [11]:
with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = json.load(f)

dish_config_json["tm_data_sources"][0] = "car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"
dish_config_json["tm_data_filepath"] = "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"

print(f"dish_config_json file contents: \n{dish_config_json}")
tmc_central_node.LoadDishCfg(json.dumps(dish_config_json))

sleep(2)
print(f"TMC CSP Master's Dish Vcc Config attribute value: \n{tmc_csp_master.dishVccConfig}")
print(
    f"\nTMC CSP Master's Source Dish Vcc Config attribute value: \n{tmc_csp_master.sourceDishVccConfig}"
)

dish_config_json file contents: 
{'interface': 'https://schema.skao.int/ska-mid-cbf-initsysparam/1.0', 'tm_data_sources': ['car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata'], 'tm_data_filepath': 'instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json'}
TMC CSP Master's Dish Vcc Config attribute value: 
{"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "dish_parameters": {"SKA001": {"vcc": 1, "k": 100}, "SKA036": {"vcc": 2, "k": 618}, "SKA063": {"vcc": 3, "k": 1220}, "SKA100": {"vcc": 4, "k": 1815}}}

TMC CSP Master's Source Dish Vcc Config attribute value: 
{"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "tm_data_sources": ["car://gitlab.com/ska-telescope/ska-telmodel-data?main#tmdata"], "tm_data_filepath": "instrument/ska1_mid_psi/ska-mid-cbf-system-parameters.json"}


### Turn the Telescope On

In [12]:
print("Running the TelescopeOn command")
tmc_central_node.TelescopeOn()

while int(tmc_central_node.telescopeState) != 0:
    print("Waiting for Telescope to come ON")
    sleep(5)
    
print("YAY Telecope is ON")


Running the TelescopeOn command
Waiting for Telescope to come ON
Waiting for Telescope to come ON
Waiting for Telescope to come ON
Waiting for Telescope to come ON
Waiting for Telescope to come ON
Waiting for Telescope to come ON
Waiting for Telescope to come ON
YAY Telecope is ON


In [13]:
print("Verifying the states:")
print(f"  TMC Central Node State: {tmc_central_node.State()}")
print(f"  CSP Control State: {csp_control.State()}")
print(f"  CBF Controller State: {cbf_controller.State()}")
print(f"  TMC Subarray State: {tmc_subarray.State()}")

Verifying the states:
  TMC Central Node State: ON
  CSP Control State: ON
  CBF Controller State: ON
  TMC Subarray State: ON


### Assign Resources

In [14]:
print(sdp_subarray.state())

sleep(3)

print(
    "Running the AssignResources command: subarray obsstate should go to Idle and receptor IDs should be assigned"
)

with open(ASSIGN_RESOURCES_FILE, encoding="utf-8") as f:
    assign_resources_json = json.load(f)
    assign_resources_json["dish"]["receptor_ids"] = RECEPTORS
    assign_resources_json["sdp"]["resources"]["receptors"] = RECEPTORS


print(f"\nassign_resources_json file contents: \n{assign_resources_json}")

tmc_subarray.AssignResources(json.dumps(assign_resources_json))

sleep(2)
print(f"\nCBF Subarray Observation State: {tmc_subarray.obsState}")
print(f"CBF Subarray Receptors : {cbf_subarray.receptors}")

ON
Running the AssignResources command: subarray obsstate should go to Idle and receptor IDs should be assigned

assign_resources_json file contents: 
{'interface': 'https://schema.skao.int/ska-tmc-assignresources/2.1', 'transaction_id': 'txn-....-00001', 'subarray_id': 1, 'dish': {'receptor_ids': ['SKA001']}, 'sdp': {'interface': 'https://schema.skao.int/ska-sdp-assignres/0.4', 'resources': {'csp_links': [1, 2, 3, 4], 'receptors': ['SKA001'], 'receive_nodes': 1}, 'execution_block': {'eb_id': 'eb-test-20210630-00059', 'context': {}, 'max_length': 21600.0, 'channels': [{'channels_id': 'vis_channels', 'spectral_windows': [{'spectral_window_id': 'fsp_1_channels', 'count': 14880, 'start': 0, 'stride': 1, 'freq_min': 296000000.0, 'freq_max': 496000000.0, 'link_map': [[0, 0], [200, 1], [744, 2], [944, 3]]}]}], 'polarisations': [{'polarisations_id': 'all', 'corr_type': ['XX', 'XY', 'YX', 'YY']}], 'fields': [{'field_id': 'field_a', 'phase_dir': {'ra': [2.711325], 'dec': [-0.01328889], 'referen

In [15]:
from astropy.time import Time
import time
from tango import DeviceProxy

dish_deployments = [f"mid-dish/dish-manager/SKA001"]


def get_tai_from_unix_s(unix_s: float) -> float:
    """
    Calculate atomic time in seconds from unix time in seconds.

    :param unix_s: Unix time in seconds

    :return: atomic time (tai) in seconds
    """
    astropy_time_utc = Time(unix_s, format="unix")
    return astropy_time_utc.unix_tai


for dish in dish_deployments:
    dish_manager_proxy = DeviceProxy("mid-dish/dish-manager/SKA001")
    current_pointing = dish_manager_proxy.achievedPointing
    current_az = current_pointing[1]
    current_el = current_pointing[2]

    current_time_tai_s = get_tai_from_unix_s(time.time() + 120)

    # Directions to move values
    az_dir = 1 if current_az < 350 else -1
    el_dir = 1 if current_el < 80 else -1

    track_table = [
        current_time_tai_s + 3,
        current_az + 1 * az_dir,
        current_el + 1 * el_dir,
        current_time_tai_s + 5,
        current_az + 2 * az_dir,
        current_el + 2 * el_dir,
        current_time_tai_s + 7,
        current_az + 3 * az_dir,
        current_el + 3 * el_dir,
        current_time_tai_s + 9,
        current_az + 4 * az_dir,
        current_el + 4 * el_dir,
        current_time_tai_s + 11,
        current_az + 5 * az_dir,
        current_el + 5 * el_dir,
    ]

    dish_manager_proxy.programTrackTable = track_table

In [19]:
print(f"\nCBF Subarray Observation State: {tmc_subarray.obsState}")


CBF Subarray Observation State: 1


### Configure Scan

In [24]:
print("Running the Configure command: subarray obsstate should go to Ready")

with open(CONFIGURE_SCAN_FILE, encoding="utf-8") as f:
    configure_scan_json = json.load(f)

print(f"\nconfigure_scan_json file contents: \n{configure_scan_json}")

print(json.dumps(configure_scan_json))

tmc_subarray.Configure(json.dumps(configure_scan_json))
    
sleep(12)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

Running the Configure command: subarray obsstate should go to Ready

configure_scan_json file contents: 
{'interface': 'https://schema.skao.int/ska-tmc-configure/2.0', 'pointing': {'target': {'reference_frame': 'ICRS', 'target_name': 'Polaris Australis', 'ra': '21:08:47.92', 'dec': '-88:57:22.9'}}, 'sdp': {'interface': 'https://schema.skao.int/ska-sdp-configure/0.4', 'scan_type': 'science'}, 'csp': {'interface': 'https://schema.skao.int/ska-csp-configure/2.4', 'subarray': {'subarray_name': 'Receptor(s)'}, 'common': {'config_id': '1 receptor, band 1, 2 FSP, no options', 'subarray_id': 1, 'frequency_band': '1'}, 'cbf': {'fsp': [{'fsp_id': 1, 'function_mode': 'CORR', 'frequency_slice_id': 1, 'zoom_factor': 0, 'integration_factor': 10, 'output_link_map': [[0, 1]], 'channel_offset': 0, 'zoom_window_tuning': 450000}]}}, 'tmc': {'scan_duration': 920.0}, 'dish': {'receiver_band': '1'}}
{"interface": "https://schema.skao.int/ska-tmc-configure/2.0", "pointing": {"target": {"reference_frame": "IC

DevFailed: DevFailed[
DevError[
    desc = ska_tango_base.faults.StateModelError: Configure command not permitted in observation state RESOURCING
           
  origin = Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tango/device_server.py", line 85, in wrapper
    return get_worker().execute(fn, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/tango/green.py", line 101, in execute
    return fn(*args, **kwargs)
  File "/app/src/ska_tmc_subarraynode/subarray_node.py", line 407, in is_Configure_allowed
    super().is_Configure_allowed()
  File "/usr/local/lib/python3.10/dist-packages/ska_tango_base/subarray/subarray_device.py", line 701, in is_Configure_allowed
    raise StateModelError(
ska_tango_base.faults.StateModelError: Configure command not permitted in observation state RESOURCING

  reason = PyDs_PythonError
severity = ERR]

DevError[
    desc = Failed to execute command_inout on device ska_mid/tm_subarray_node/1, command Confi

In [ ]:
print(f"\nCBF Subarray Observation State: {cbf_subarray.obsState}")

### Scan

DO NOT USE FOR NOW

In [ ]:
print("Running the Scan command: subarray obsstate should go to Scanning")

with open(SCAN_FILE, encoding="utf-8") as f:
    scan_json = f.read()

print(f"\nscan_json file contents: \n{scan_json}")

tmc_subarray.Scan(scan_json)

sleep(2)
print(f"\nCBF Subarray Observation State: {cbf_subarray.obsState}")

TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING

In [ ]:
with open("../../data/mid_telescope/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    sdp_scan = d["sdp"]
    print(sdp_scan)
    sdp_subarray_leaf_node_dp.scan(json.dumps(sdp_scan))

sleep(10)
print(sdp_subarray_leaf_node_dp.sdpSubarrayObsState)

with open("../../data/mid_telescope/tmc/scan.json", "r", encoding="utf-8") as json_data:
    d = json.load(json_data)
    csp_scan = d["csp"]
    print(csp_scan)
    csp_subarray_leaf_node_dp.scan(json.dumps(csp_scan))

sleep(2)
print(csp_subarray_leaf_node_dp.cspSubarrayObsState)

### END SCAN


DO NOT USE FOR NOW SINCE TMC SUBARRAY IS STUCK IN CONFIGURING

In [ ]:
print("Running the End Scan command: subarray obsstate should go to Ready state")

tmc_subarray.EndScan()

sleep(2)
print(f"\nTMC Subarray Observation State: {tmc_subarray.obsState}")

TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING

In [ ]:
print("Running the End Scan command: subarray obsstate should go to Ready state")

sdp_subarray_leaf_node_dp.EndScan()
sleep(2)
csp_subarray_leaf_node_dp.EndScan()
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

### END

DO NOT USE (TMC STUCK IN CONFIGURING)

In [ ]:
print("Running the End command: subarray obsstate should go to Idle state")

tmc_subarray.End()

sleep(2)
print(f"\TMC Subarray Observation State: {cbf_subarray.obsState}")

TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING

In [ ]:
print("Running the End command: subarray obsstate should go to Idle state")

sdp_subarray_leaf_node_dp.End()
sleep(2)
csp_subarray_leaf_node_dp.End()

sleep(2)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

### Release Resources

DO NOT USE (TMC STUCK IN CONFIGURING)

In [25]:
print(
    "Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty"
)

tmc_subarray.ReleaseAllResources()

sleep(2)
print(f"TMC Subarray Observation State: {tmc_subarray.obsState}")

Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty


DevFailed: DevFailed[
DevError[
    desc = ska_tango_base.faults.StateModelError: ReleaseAllResources command not permitted in observation state RESOURCING
           
  origin = Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tango/device_server.py", line 85, in wrapper
    return get_worker().execute(fn, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/tango/green.py", line 101, in execute
    return fn(*args, **kwargs)
  File "/app/src/ska_tmc_subarraynode/subarray_node.py", line 374, in is_ReleaseAllResources_allowed
    super().is_ReleaseAllResources_allowed()
  File "/usr/local/lib/python3.10/dist-packages/ska_tango_base/subarray/subarray_device.py", line 661, in is_ReleaseAllResources_allowed
    raise StateModelError(
ska_tango_base.faults.StateModelError: ReleaseAllResources command not permitted in observation state RESOURCING

  reason = PyDs_PythonError
severity = ERR]

DevError[
    desc = Failed to execute command_inout o

TMC LEAF NODE HACK TO GET AROUND TMC SUBARRAY STUCK IN CONFIGURING

In [ ]:
print(
    "Running the Release All Resources command: subarray obsstate should go to Empty state and receptor IDs should be empty"
)

sdp_subarray_leaf_node_dp.ReleaseAllResources()
sleep(2)
csp_subarray_leaf_node_dp.ReleaseAllResources()

sleep(2)
print(f"SDP Subarray Observation State: {sdp_subarray_leaf_node_dp.sdpSubarrayObsState}")
print(f"CSP Subarray Observation State: {csp_subarray_leaf_node_dp.cspSubarrayObsState}")

### Telescope Off

In [ ]:
print("Running the TelescopeOff command")

tmc_central_node.TelescopeOff()

sleep(5)
print(f"TMC Central Node State: {tmc_central_node.State()}")
print(f"CSP Control State: {csp_control.State()}")
print(f"CBF Controller State: {cbf_controller.State()}")